In [77]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fractions import Fraction as frac
from decimal import Decimal
import regex as re

In [2]:
url = 'https://maplestory.nexon.com/Guide/OtherProbability/cube/red'


In [132]:
def process_df(df,first=True,firstweights=None,primeprob=None):
    split = df.iloc[:,0].str.split(':',expand=True)
    split.columns = ['Stat','Value']
    split.Stat.loc[split.Value.str.contains('%',na=False) & (split.Stat != '모든  스킬의 재사용 대기시간 ')] = split.Stat.loc[split.Value.str.contains('%',na=False)] + "%"
    split.Value = split.Value.str.extract('([0-9]+)').astype('int',errors='ignore')
    split.fillna({'Value':1},inplace=True)
    split.Stat = split.Stat.str.strip()
    probs = df.iloc[:,1].apply(lambda x: float(x.strip('%'))/100)
    if first:
        reduced = probs/min(probs)
        weights = reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100) * max(reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100).denominator))).astype('int')
    else:
        weights = pd.Series(index=probs.index)
        weights[-len(firstweights):] = firstweights
        weights[:-len(firstweights)] = probs[:-len(firstweights)] / primeprob * sum(firstweights)
        weights = weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20) * max(weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20).denominator)))
    split['Weight'] = weights
    return split



In [87]:
def get_cube_weights(url):
    html = requests.get(url).content
    df_list = pd.read_html(html, match='STR')
    soup = BeautifulSoup(html, 'html.parser')
    table_properties = soup.find_all('div',{'class': 'cube_option'})
    cube_df = []
    for i in range(0,len(df_list)):
        metadata = re.findall('<span>(.+)</span>',str(table_properties[i]))
        if metadata[0] == '레어':
            no_rare_lines = len(df_list[i].iloc[:,0:2].dropna())
            normal_lines = df_list[i].iloc[:-no_rare_lines,2:4].dropna()
            minidf = process_df(normal_lines)
            minidf['Grade'] = 'Normal'
            minidf['Equip'] = metadata[1]
            minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
            minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
            cube_df.append(minidf)
        primelines = df_list[i].iloc[:,0:2].dropna()
        minidf = process_df(primelines)
        minidf['Grade'] = metadata[0]
        minidf['Equip'] = metadata[1]
        minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
        minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
        cube_df.append(minidf)

    alldf = pd.concat(cube_df,ignore_index=True)
    return alldf

In [139]:
cubedict = {'Red':'https://maplestory.nexon.com/Guide/OtherProbability/cube/red',
           'Black': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/black',
           'Additional': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/addi'}
df = []
for cube,url in cubedict.items():
    cubeweights = get_cube_weights(url)
    cubeweights['Cube'] = cube
    df.append(cubeweights)

masterlist = pd.concat(df,ignore_index=True)
masterlist.to_csv('weights_kr.csv',index=False)

In [142]:
# koreancolumns = masterlist[['Equip','Grade','Stat']]
# koreanwords = []
# for i in koreancolumns.columns:
#     unique = koreancolumns[i].unique()
#     koreanwords += [x for x in unique]
# with open('koreanwords.csv','w+') as f:
#     for j in koreanwords:
#         f.write('"'+j+'"\n')

dictionary = pd.read_excel('translations.xlsx')
def translate(df,fromlang='kr',tolang='en'):
    translation_map = dictionary.set_index(fromlang)[tolang].to_dict()
    df2 = df.copy()
    for i in df.columns:
        df2[i] = df[i].map(lambda x: translation_map.get(x,x))
    return df2

translate(masterlist).to

,Equip,Level Requirement,Grade,Stat,Value,Weight,Cube
0,Weapon,120,Normal,STR,6,3,Red
1,Weapon,120,Normal,DEX,6,3,Red
2,Weapon,120,Normal,INT,6,3,Red
3,Weapon,120,Normal,LUK,6,3,Red
4,Weapon,120,Normal,Max HP,60,3,Red
...,...,...,...,...,...,...,...
4285,Heart,100,Legendary,LUK per 10 character levels,2,2,Additional
4286,Heart,100,Legendary,All Elemental Resistances %,5,1,Additional
4287,Heart,100,Legendary,HP Recovery % (on items and skills),30,3,Additional
4288,Heart,100,Legendary,Meso Obtained %,5,3,Additional
